In [31]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
# import eli5
import pickle

sns.set()
warnings.filterwarnings('ignore')

In [32]:
from sklearn.feature_selection import mutual_info_regression, SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier, RandomForestRegressor, RandomForestClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, scale, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_absolute_percentage_error, mean_squared_error, roc_auc_score, log_loss, precision_recall_fscore_support, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from imblearn.over_sampling import RandomOverSampler, SMOTE
# from eli5.sklearn import PermutationImportance
from pprint import pprint

from xgboost import XGBRegressor, XGBClassifier

---

In [33]:
pd.set_option('display.max_columns', None)

---

**loading data + X, y split**

In [34]:
path_x1="C:/Users/DELL/Desktop/flask_project/X1.csv"
path_x2="C:/Users/DELL/Desktop/flask_project/X2.csv"
path_y1="C:/Users/DELL/Desktop/flask_project/Y1.csv"
path_y2="C:/Users/DELL/Desktop/flask_project/Y2.csv"

x1=pd.read_csv(path_x1)
y1=pd.read_csv(path_y1)
x2=pd.read_csv(path_x2)
y2=pd.read_csv(path_y2)

In [35]:
x1.drop('Unnamed: 0',inplace=True,axis=1)
x2.drop('Unnamed: 0',inplace=True,axis=1)
y1.drop('Unnamed: 0',inplace=True,axis=1)
y2.drop('Unnamed: 0',inplace=True,axis=1)

In [36]:
y1.head()

,LoanStatus
0,1
1,1
2,1
3,1
4,1


In [37]:
y2.head()

,EMI,ROI,EligibleLoanAmount
0,1489.150000,0.136442,11099.999999
1,825.275000,0.215686,7499.999999
2,132.500000,0.116998,6000.000001
3,12.719840,0.125492,8700.000001
4,66.637575,0.241102,19800.000000


In [38]:
y1_loan_status = y1['LoanStatus']
y2_EMI= y2['EMI']
y2_ROI= y2['ROI']
y2_EligibleLoanAmount= y2['EligibleLoanAmount']

In [39]:
x1.head()

,BorrowerAPR,CreditScoreRangeLower,CreditScoreRangeUpper,LP_CustomerPrincipalPayments,EstimatedReturn,LenderYield,LP_CustomerPayments,EstimatedLoss,BorrowerRate
0,0.313647,0.727273,0.727273,0.269286,0.622761,0.294527,0.280029,0.247189,0.317588
1,0.546030,0.545455,0.545455,0.085743,0.622761,0.497512,0.102911,0.247189,0.552764
2,0.284325,0.727273,0.727273,0.028571,0.622761,0.263682,0.024911,0.247189,0.266332
3,0.342376,0.772727,0.772727,0.002533,0.550386,0.285572,0.003394,0.131543,0.288442
4,0.694875,0.795455,0.795455,0.005993,0.658877,0.632239,0.012860,0.443367,0.638593


In [40]:
x2.head()

,LP_CustomerPayments,𝐿𝑜𝑎𝑛𝐷𝑎𝑡𝑒𝑦𝑒𝑎𝑟,month,𝑀𝑎𝑡𝑢𝑟𝑖𝑡𝑦𝐷𝑎𝑡𝑂𝑟𝑖𝑔𝑖𝑛𝑎𝑙𝑦𝑒𝑎𝑟,𝑀𝑎𝑡𝑢𝑟𝑖𝑡𝑦𝐷𝑎𝑡𝑒𝑂𝑟𝑖𝑔𝑖𝑛𝑎𝑙𝑚𝑜𝑛𝑡ℎ,LP_CustomerPrincipalPayments,TradesNeverDelinquent (percentage),AvailableBankcardCredit,TotalInquiries,InterestAmount,TotalAmount,StatedMonthlyIncome,DebtToIncomeRatio,IncomeVerifiable,LoanOriginalAmount,BorrowerRate,LoanTenure
0,11396.1400,1970,1,2007,9,9425.00,0.810000,1500.000000,3.0,1489.150,10914.150,3083.333333,0.17,1,9425.0,0.1580,436
1,4186.6300,1970,1,2007,1,3001.00,0.857548,10761.775498,1.0,825.275,3826.275,2083.333333,0.06,1,3001.0,0.2750,444
2,1011.6500,1970,1,2007,10,1000.00,0.400000,2580.000000,5.0,132.500,1132.500,1666.666667,0.27,1,1000.0,0.1325,435
3,135.8174,1970,1,2014,1,88.64,1.000000,3626.000000,4.0,574.000,4574.000,2416.666667,0.18,1,4000.0,0.1435,528
4,521.1300,1970,1,2012,4,209.75,0.940000,178.000000,1.0,1270.800,5270.800,5500.000000,0.49,1,4000.0,0.3177,501


In [41]:
y2_EMI=y2_EMI.value_counts()
print(y2_EMI)
y2_EOI=y2_ROI.value_counts()
print(y2_ROI)
y2_EligibleLoanAmount=y2_EligibleLoanAmount.value_counts()
print(y2_EligibleLoanAmount)


0.000000       515
1270.800000    380
1279.600000     93
635.400000      74
1050.000000     70
              ... 
2564.161230      1
360.958872       1
1794.150000      1
1826.974692      1
659.750000       1
Name: EMI, Length: 29828, dtype: int64
0        0.136442
1        0.215686
2        0.116998
3        0.125492
4        0.241102
           ...   
42353    0.097473
42354    0.114653
42355    0.172802
42356    0.224806
42357    0.208798
Name: ROI, Length: 42358, dtype: float64
18000.000000    1211
15000.000001    1137
11999.999999    1096
9000.000000     1025
13500.000000     880
                ... 
34762.500000       1
11998.500001       1
11716.800001       1
6122.400001        1
4908.300001        1
Name: EligibleLoanAmount, Length: 7667, dtype: int64


---

**train, test split**

In [42]:
print(x1.shape)
print(y1.shape)

(57361, 9)
(57361, 1)


In [43]:
print(x2.shape)
print(y2.shape)

(42358, 17)
(42358, 3)


In [44]:
y2 = y2.fillna(0)

In [45]:
X_train1, X_test1, y_train, y_test = train_test_split(x1, y1, test_size=0.20, random_state=0)

yLoanstates_train = y_train
yLoanstates_test = y_test

X_train2, X_test2, y_train, y_test = train_test_split(x2, y2, test_size=0.20, random_state=0)


yEMI_train = y_train['EMI']
yROI_train = y_train['ROI']
yEligibleLoanAmount_train = y_train['EligibleLoanAmount']

yEMI_test = y_test['EMI']
yROI_test = y_test['ROI']
yEligibleLoanAmount_test = y_test['EligibleLoanAmount']




---

## data preprocessing

In [46]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

# Define the oversampling and scaling transformers
over = RandomOverSampler(random_state=0)
smote = SMOTE()
stdscaler = StandardScaler()
scaler = MinMaxScaler()


pca = PCA(n_components=2)

# Define the pipeline
steps=[("scaling",StandardScaler()),
      ("PCA",PCA(n_components=3)),
      ]
pipeline=Pipeline(steps)


---

## classification

**classifier learning**

In [47]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor

In [48]:
xgb_regressor = xgb.XGBRegressor()
ls = Lasso(alpha=0.1)
lr = LinearRegression()


In [49]:
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report

# Create the individual classifier
xgb_classifier = XGBClassifier()

# Create the stacking classifier
estimators = [('XGB', xgb_classifier)]
clf = StackingClassifier(estimators=estimators)

# Create the pipeline
model_classifier = Pipeline([
    ('stdscaler', StandardScaler()),
    ('pca', PCA()),
    ('classifier', clf)
])

# Fit the classifier on the training data
model_classifier.fit(X_train1, yLoanstates_train)

# Make predictions on the test data
pred = model_classifier.predict(X_test1)

# Calculate and print the test accuracy
print('Test accuracy =', round(accuracy_score(yLoanstates_test, pred) * 100, 2), '%')


Test accuracy = 91.75 %


In [50]:
print(classification_report(yLoanstates_test, pred, digits=3))

              precision    recall  f1-score   support

           0      0.860     0.818     0.838      2998
           1      0.937     0.953     0.945      8475

    accuracy                          0.918     11473
   macro avg      0.898     0.885     0.891     11473
weighted avg      0.917     0.918     0.917     11473



In [51]:
X_test1.head()

,BorrowerAPR,CreditScoreRangeLower,CreditScoreRangeUpper,LP_CustomerPrincipalPayments,EstimatedReturn,LenderYield,LP_CustomerPayments,EstimatedLoss,BorrowerRate
44454,0.156952,0.840909,0.840909,0.178571,0.513937,0.164179,0.173932,0.029355,0.165829
16559,0.394080,0.704545,0.704545,0.007309,0.622761,0.404975,0.009625,0.247189,0.398995
43181,0.575826,0.659091,0.659091,0.033389,0.622761,0.587065,0.054574,0.247189,0.582915
4108,0.565505,0.704545,0.704545,0.033942,0.683962,0.512239,0.059243,0.277209,0.517387
18042,0.530074,0.795455,0.795455,0.242857,0.700901,0.495323,0.259795,0.232899,0.500302


In [52]:
X_test1.to_csv('filename.csv', index=False)

In [53]:
import pickle
filename='classification_model'
pickle.dump(model_classifier,open(filename,'wb'))

In [54]:
load_model=pickle.load(open(filename,'rb'))

In [55]:
load_model.predict([[0.156952,0.840909,0.840909,0.178571,0.513937,0.164179,0.173932,0.029355,0.165829]])

array([1], dtype=int64)

**regression**

In [56]:
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import r2_score

In [57]:
#Buid Pipeline
estimators = [('Lasso', ls), ('linearRegression', lr)]
final_estimator = GradientBoostingRegressor()
clf = StackingRegressor(estimators=estimators, final_estimator=final_estimator)

model_regressor=Pipeline([
('stdscaler', stdscaler),
('classifier', clf)
])

EMI

In [58]:
model_regressor.fit(X_train2, yEMI_train)
pred = model_regressor.predict(X_test2)
print('test accuracy = ', round(r2_score(yEMI_test, pred)*100, 2), '%')

test accuracy =  96.3 %


ROI

In [59]:

model_regressor.fit(X_train2, yROI_train)
pred = model_regressor.predict(X_test2)
print('test accuracy = ', round(r2_score(yROI_test, pred)*100, 2), '%')

test accuracy =  99.95 %


ELA

In [60]:
model_regressor.fit(X_train2, yEligibleLoanAmount_train)
pred = model_regressor.predict(X_test2)
print('test accuracy = ', round(r2_score(yEligibleLoanAmount_test, pred)*100, 2), '%')

test accuracy =  99.32 %


---